<a href="https://colab.research.google.com/github/kdavisong/Proyectos/blob/main/PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MOVIES RECOMENDATIONS
Kevin Davison Galvis

In [75]:
import numpy as np
import pandas as pd
from time import strftime
from datetime import datetime
import ast
from ast import literal_eval

In [76]:
#1 Llemos dataset base y generamos una copia para desanidar y modificar
df = pd.read_csv("movies_dataset.csv")
iso_639_1 = pd.read_csv("iso_639_1.csv")
df2 = df
c = pd.read_csv("credits.csv")
Crew = pd.DataFrame()
Saga = pd.DataFrame()


C:\Users\KEVIN\AppData\Local\Temp\ipykernel_8980\2708928596.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_dataset.csv")


In [77]:
###3_Rellenamos los valores vacios
df2["belongs_to_collection"] = df2["belongs_to_collection"].fillna("{'id': 'NaN','name': 'NaN','poster_path': 'NaN','backdrop_path': 'NaN'}")
df2["genres"] = df2["genres"].fillna("[]")
df2["production_companies"] = df2["production_companies"].fillna("[]")
df2["production_countries"] = df2["production_countries"].fillna("[]")
df2["spoken_languages"] = df2["spoken_languages"].fillna("[]")
df2["revenue"] = df2["revenue"].fillna(0)
df2["budget"] = df2["budget"].fillna(0)
c["crew"] = c["crew"].fillna("[]")

In [78]:
###4Se realizan conversiones en los casos donde se lee los datos de la columna como str
#df2["belongs_to_collection"] = df2["belongs_to_collection"].apply(literal_eval)

In [79]:
#Se aplica la función literal eval para los casos donde se leen listas o dic como str
df2.loc[:,"belongs_to_collection"] = df2["belongs_to_collection"].apply(literal_eval)
df2.loc[:, "genres"] = df2["genres"].apply(literal_eval)
df2.loc[:, "production_companies"] = df2["production_companies"].apply(literal_eval)
df2.loc[:, "production_countries"] = df2["production_countries"].apply(literal_eval)
df2.loc[:, "spoken_languages"] = df2["spoken_languages"].apply(literal_eval)
c.loc[:, "crew"] = c["crew"].apply(literal_eval)

In [80]:
#7 Se eliminan columnas innecesarias
df2 = df2.drop(["video", "imdb_id", "adult", "homepage", "original_title", "poster_path"], axis = 1)

In [81]:
mask = df2['belongs_to_collection'].apply(lambda x: isinstance(x, float) and not np.isnan(x))
df2.drop(df2[mask].index, inplace=True)
mask = df2['production_countries'].apply(lambda x: isinstance(x, float) and not np.isnan(x))
df2.drop(df2[mask].index, inplace=True)

In [82]:
mascara = df2["production_countries"].apply(lambda x: isinstance(x, float))
#mascara = df2["production_companies"].apply(lambda x: isinstance(x, float))
floatvalues = df2[mascara]
floatvalues

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [83]:
#8_Se crea nuevos dataframes para ingresarle la información desanidada
Saga["id_movie"] = df2["id"]
Production_Countries = pd.DataFrame(columns = ["iso_3166_1","name"])
Production_Countries["id_movie"] = df2["id"]
Genres = pd.DataFrame(columns = ["id","name"])
Genres["id_movie"] = df2["id"]
Productoras = pd.DataFrame(columns =["id","name"])
Productoras["id_movie"] = df2["id"]
Idiomas = pd.DataFrame(columns = ["iso_3166_1","name"])
Idiomas["id_movie"] = df2["id"]
Crew["crew"] = c["crew"]
Crew["id_movie"] = c["id"]

In [84]:
mascara = df2["production_countries"].apply(lambda x: isinstance(x, float))
#mascara = df2["production_companies"].apply(lambda x: isinstance(x, float))
floatvalues = df2[mascara]
floatvalues

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [85]:
#Se copia columna original para desanidar
Saga["belongs_to_collection"] = df2["belongs_to_collection"]
Productoras["production_companies"] = df2["production_companies"]
Production_Countries["production_countries"] = df2["production_countries"]
Genres["genres"] = df2["genres"]
Idiomas["spoken_languages"] = df2["spoken_languages"]

In [86]:
#generamos nuevo df para desanidar columna production companies
Productoras["Numero_productoras"] = Productoras["production_companies"].apply(lambda x: len(x) if not isinstance(x, bool) else None)
Production_Countries["number_of_countries"] = Production_Countries["production_countries"].apply(lambda x: len(x) if not isinstance(x, float) else None)
Genres["number_of_genres"] = Genres["genres"].apply(lambda x: len(x))
Idiomas["number_of_languages"] = Idiomas["spoken_languages"].apply(lambda x: len(x))

In [87]:
mascara = Saga["belongs_to_collection"].apply(lambda x: isinstance(x, float))
floatvalues = Saga[mascara]
floatvalues

,id_movie,belongs_to_collection


In [88]:
def desanidar_columna_dic(data, column_name):
    filas_desempacadas = []
    for i, row in data.iterrows():
        id_value = row['id_movie']
        diccionario_data = row[column_name]
        fila = {'id_movie': id_value}
        fila.update(diccionario_data)
        filas_desempacadas.append(fila)
    return pd.DataFrame(filas_desempacadas)

In [89]:
def desanidar_columna(data, column_name):
    filas_desempacadas = []
    for i, row in data.iterrows():
        id_value = row['id_movie']
        product = row[column_name]
        for diccionario in product:
            fila = {'id_movie': id_value}
            fila.update(diccionario)
            filas_desempacadas.append(fila)
    return pd.DataFrame(filas_desempacadas)

In [90]:
##DESEMPACANDO Y GENERANDO LISTAS INDEPENDIENTES DEJANDO EL ID DE PELICULA COMO CAMPO RELACIONAL
Saga = desanidar_columna_dic(Saga, "belongs_to_collection")

Genres = desanidar_columna(Genres, "genres")

Idiomas = desanidar_columna(Idiomas, "spoken_languages")
Idiomas = pd.merge(Idiomas, iso_639_1, on = "iso_639_1")
Idiomas = Idiomas.drop(["name"], axis = 1)

Production_Countries = desanidar_columna(Production_Countries, "production_countries")

Productoras = desanidar_columna(Productoras, "production_companies")

Crew = desanidar_columna(Crew, "crew")

In [91]:
###Se generan nuevas columnas de reñación de ganancia y presupuesto "return" y el año de lanzamiento
df2["return"] = df2["revenue"] / df2["budget"].astype(float)
df2["release_year"] = df2["release_date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").strftime("%Y") if not pd.isnull(x) else x)
df2["release_year"] = df2["release_year"].fillna(0)
df2.drop(["genres", "production_countries", "production_companies"], axis = 1, inplace = True)
df2 = df2.rename(columns ={"id":"id_movie"})


In [92]:
#Generamos csv de nuestras tablas desanidadas
Genres.to_csv("genres.csv")
Idiomas.to_csv("Idiomas.csv")
Production_Countries.to_csv("Production_countries.csv")
Productoras.to_csv("Productoras.csv")
df2.to_csv("movies_data.csv")
Crew.to_csv("Crew.csv")
Saga.to_csv("Saga.csv")
